In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

In [ ]:
registry_patient_selection_path = '/Users/jk1/temp/opsum_extraction_output/high_frequency_data_patient_selection_with_details.csv'
output_path = '/Users/jk1/temp/opsum_extration_output'

In [ ]:
registry_data_df = pd.read_csv(registry_patient_selection_path, dtype=str)

In [ ]:
# registry_data_df['registry_stroke_onset'] = pd.to_datetime(pd.to_datetime(registry_data_df['Stroke onset date'], format='%Y%m%d').dt.strftime('%d-%m-%Y') \
#                                         + ' ' + registry_data_df['Stroke onset time'], format='%d-%m-%Y %H:%M', ).dt.strftime('%d.%m.%Y %H:%M')
#
# registry_data_df['registry_admission_date'] = pd.to_datetime(pd.to_datetime(registry_data_df['Arrival at hospital'], format='%Y%m%d').dt.strftime('%d-%m-%Y') \
#                                         + ' ' + registry_data_df['Arrival time'], format='%d-%m-%Y %H:%M').dt.strftime('%d.%m.%Y %H:%M')

In [ ]:
from preprocessing.utils import create_registry_case_identification_column

registry_data_df['case_admission_id'] = create_registry_case_identification_column(registry_data_df)

In [ ]:
len(registry_data_df['case_admission_id'].unique())

In [ ]:
import pandas as pd
eds_df = pd.read_csv('/Users/jk1/stroke_datasets/stroke_unit_dataset/per_value/Extraction_20220815/eds_j1.csv', delimiter=';', dtype=str)

In [ ]:
from preprocessing.utils import create_ehr_case_identification_column

eds_df['case_admission_id'] = create_ehr_case_identification_column(eds_df)

In [ ]:
len(eds_df['case_admission_id'].unique())

In [ ]:
from preprocessing.patient_selection.filter_ehr_patients import filter_ehr_patients

filtered_eds_df = filter_ehr_patients(eds_df, registry_patient_selection_path)
len(filtered_eds_df['case_admission_id'].unique())

In [ ]:
filtered_eds_df[filtered_eds_df.patient_id == '97738228'][['patient_id', 'patient_id_manual', 'eds_final_patient_id', 'eds_end_4digit', 'eds_manual', 'eds_final_id']]

In [ ]:
# find number of case_admission_ids in registry and eds
len(set(registry_data_df['case_admission_id'].unique()).intersection(set(filtered_eds_df['case_admission_id'].unique())))

In [ ]:
# find patients only present in registry and missing from eds
set(registry_data_df['case_admission_id'].unique()) - set(filtered_eds_df['case_admission_id'].unique())

In [ ]:
# find patients only present in eds
set(eds_df['case_admission_id'].unique()).difference(set(registry_data_df['case_admission_id'].unique()))

In [ ]:
eds_df.rename(columns={'eds_end_4digit': 'EDS_last_4_digits'}, inplace=True)

In [ ]:
eds_df.head()

In [ ]:
registry_data_df.head()

Restrict to patient selection

In [ ]:
restricted_eds_df = eds_df[eds_df['case_admission_id'].isin(registry_data_df['case_admission_id'])]


In [ ]:
joined_df = pd.merge(registry_data_df, restricted_eds_df, on=['case_admission_id'], how='left')

In [ ]:
joined_df.head()

##### Impression: For correspondence between registry and DPI data use case id (patient id + EDS last 4 digits) with begin date.

Notes:
- In DPI data, for the same case id, multiple begin dates are found -> restrict analysis to begin date found in stroke registry.

# Find intra-hospital stroke patients / patients with a stroke after hospital admission


In [ ]:
joined_df['delta_onset_eds_date_from_in_days'] = (pd.to_datetime(joined_df['Onset date'], format='%Y%m%d') - pd.to_datetime(joined_df['date_from'], format='%d.%m.%Y %H:%M')).dt.days

In [ ]:
joined_df[['case_admission_id','date_from', 'Onset date', 'delta_onset_eds_date_from_in_days']]

In [ ]:
# find patients where admission date is correct but there is a difference between onset date and date_from
sub_acute_stroke_admissions = joined_df[(joined_df['delta_onset_eds_date_from_in_days'] <= -7)
          & (~joined_df['case_admission_id'].isin(differing_starting_date_df['case_admission_id']))
          & (~joined_df['case_admission_id'].isin(minor_differing_starting_date_df['case_admission_id']))]

In [ ]:
sub_acute_stroke_admissions[['case_admission_id','date_from', 'registry_stroke_onset', 'delta_onset_eds_date_from_in_days']]

Impression: some admissions may be sub-acute, and some due to a glitch in onset date

In [ ]:
stroke_after_7_day_admission = joined_df[(joined_df['delta_onset_eds_date_from_in_days'] > 7)
          & (~joined_df['case_admission_id'].isin(differing_starting_date_df['case_admission_id']))
          & (~joined_df['case_admission_id'].isin(minor_differing_starting_date_df['case_admission_id']))]

In [ ]:
stroke_after_7_day_admission[['case_admission_id','date_from', 'registry_stroke_onset', 'delta_onset_eds_date_from_in_days', 'Referral']]

In [ ]:
stroke_after_7_day_admission['Referral'].value_counts()

In [ ]:
stroke_after_7_day_admission.drop(['Unnamed: 0', 'Onset date', 'Onset time', 'nr', 'patient_id_y',
       'EDS_last_4_digits_y', 'DOB_y', 'begin_date', 'end_date',
       'death_date', 'death_hosp', 'eds_final_id', 'eds_final_begin',
       'eds_final_end', 'eds_final_patient_id', 'eds_final_birth',
       'eds_final_death', 'eds_final_birth_str', 'date_from', 'date_to',
       'registry_arrival_to_date_from_days', 'eds_manual'], axis=1, inplace=True)

In [ ]:
# stroke_after_7_day_admission.to_csv(os.path.join(output_path, 'patients_with_stroke_after_7_days_admission.csv'), index=False)